### Титаник.

Домашнее задание.

In [2]:
import numpy as np
import pandas as pd

In [158]:
from  matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

%config InlineBackend.figure_format = 'retina'
from pylab import rcParams
rcParams['figure.figsize'] = (9, 6)

from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler

from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

###  Данные

Прочитаем тренировочные и тестовые данные.

In [4]:
train = pd.read_csv('titanic/train.csv')
test  = pd.read_csv('titanic/test.csv')

In [5]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


### Feature еngeneering

Создаем и описываем фичи.
Чтобы не дублировать операции для тренировочной тестовой выборки, объединим их:
- удалим целевой столбеци из train
- добавим флаг тестовой выборки
- проверим, что что поля совпадают.

In [125]:
# Прочитаем данные еще раз
train = pd.read_csv('titanic/train.csv')
test  = pd.read_csv('titanic/test.csv')

y_train = train.Survived
# Удалим целевую функцию из train
train.drop('Survived',axis=1,inplace=True)
# Пометим выборки
train['is_test'] = 0
test['is_test'] = 1
# Склеим
df = pd.concat([train,test])

# Заменим пол со строковой переменной на числовую
df['IsMale'] = df.Sex.replace({'male':1, 'female':0})

Создадим кучку фич на основе имени

In [126]:
df['HaveMrs'] = df.Name.str.contains('mrs.', case=False)
df.HaveMrs.replace({True:1, False : 0}, inplace=True)

df['HaveMr'] = df.Name.str.contains('mr\.', case = False)
df.HaveMr.replace({True : 1, False : 0},inplace=True)

df['HaveRev'] = df.Name.str.contains('rev\.', case = False)
df.HaveRev.replace({True : 1, False : 0},inplace=True)

df['HaveMiss'] = df.Name.str.contains('miss\.', case = False)
df.HaveMiss.replace({True : 1, False : 0},inplace=True)

df['HaveDr'] = df.Name.str.contains('dr\.', case = False)
df.HaveDr.replace({True : 1, False : 0},inplace=True)

df['HaveMaster'] = df.Name.str.contains('master\.', case = False)
df.HaveMaster.replace({True : 1, False : 0},inplace=True)

df['HaveCol'] = df.Name.str.contains('col\.', case = False)
df.HaveCol.replace({True : 1, False : 0},inplace=True)

Немного ручной работы - проверим, что регексп работает корректно создадим файл и посмотрим его.

In [127]:
sdf = df.loc[:,['Name','HaveMrs']]
sdf.sort_values('HaveMrs',ascending=False,inplace=True)
sdf.to_csv('have_mrs.csv')

Давим фичу HaveCabin - проверим у кого указана каюта.

In [128]:
df['HaveCabin'] = df.Cabin.isnull()
df.HaveCabin.replace({True : 1, False : 0},inplace=True)

In [102]:
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,is_test,IsMale,HaveMrs,HaveMr,HaveCol,HaveCabin
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,1,0,1,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,0,1,0,0,0
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,0,0,0,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,0,1,0,0,0
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,1,0,1,0,1


Все оставшивиеся текстовые поля удалим.

In [129]:
df.drop(['PassengerId','Name','Sex','Ticket','Cabin'],axis=1,inplace=True)

Поля - категогории разобьем на несколько столбцов

In [131]:
df_dummies = pd.get_dummies(df, columns=['Pclass','Embarked'])


Разделим тренировочную и тестовую выборку.

In [132]:
X_train = df_dummies[df_dummies.is_test == 0].drop('is_test',axis = 1)
X_test = df_dummies[df_dummies.is_test == 1].drop('is_test', axis = 1)

### Заполнение пустых значений. 

Пустые значения заполняем средними.
На основе обучающей выборки создаем imputer.

In [133]:
columns = X_train.columns
imputer = Imputer(missing_values='NaN', strategy='mean', axis = 0, verbose=1, copy = True)
imputer.fit(X_train)

Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=1)

In [134]:
X_train_imputed = imputer.transform(X_train)
X_train_imputed = pd.DataFrame(X_train_imputed, columns=columns)

### Нормировка значений

Для обучающей выборки создаем scaler - нормировщик.

In [135]:
scaler = StandardScaler()
scaler.fit(X_train_imputed)
X_train_i_s = scaler.transform(X_train_imputed)
X_train_i_s = pd.DataFrame(X_train_i_s, columns = columns)

 #### Заполняем пустые значения и нормируем тестовую выборку
 
Используем объекты imputer и scaler, обученные на обучающей выборке. Применяем эти объекты к тестовой выборке.

In [155]:
X_test_i_s = scaler.transform(imputer.transform(X_test))
X_test_i_s = pd.DataFrame(X_test_i_s, columns = columns)

# XGBoost

In [35]:
import xgboost
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

In [59]:
cv = KFold(n_splits=4, shuffle=True)

## Шаг 1: Зафиксируем learning_rate и параметры дерева и подберём n_estimators

Параметры:

* **max_depth**. Как указанов в таблице выше, обычно варьируется в интервале от 3 до 10 (но от задачи к задаче значения могут меняться). В качестве начального значения обычно используют 5
* **min_child_weight**. Если выборка сильно несбалансирована, то лучше выбрать значение "1". Иначе лучше выбрать значение "2" и зафиксировать
* **gamma**. Обычно выставляют значение в интервале от 0 до 0.2 и фиксируют. В дальнейшем этот параметр всегда можно затюнить отдельно
* **subsample, colsample_bytree**. Выставим 0.8 и зафиксируем. Можно также проварьировать в интервале 0.5-0.9.
* **scale_pos_weight**. Выставляется в зафисимости от соотношения классов в выборке и фиксируется

In [137]:
# посчитаем соотношение между классами
ones_ratio = y_train[y_train == 1].shape[0] * 1.0 / y_train[y_train == 0].shape[0] 
param_grid = {
    # параметры ансамбля
    'n_estimators': [10, 30, 50, 100, 200, 400, 600, 1000],
    'learning_rate': [0.1, ],
    
    # параметры дерева
    'max_depth': [5],
    'min_child_weight': [2],
    'gamma': [0.1],
    'subsample': [0.8],
    'colsample_bytree': [0.8],
    'scale_pos_weight': [ones_ratio],
    
    # параметры регуляризации
    'reg_alpha': [0.0],
    'reg_lambda': [1.0]
}

cv = KFold(n_splits=8, shuffle=True)

clf = xgboost.XGBClassifier()
gs = GridSearchCV(clf, param_grid, scoring='accuracy', cv=cv, verbose=0)

gs.fit(X_train_i_s, y_train)
best_params_1 = gs.best_estimator_.get_params()
print('Best score (accuracy): ', gs.best_score_)
print('Best params: ')
best_params_1

Best score (accuracy):  0.828282828283
Best params: 


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.8,
 'gamma': 0.1,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 5,
 'min_child_weight': 2,
 'missing': None,
 'n_estimators': 200,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0.0,
 'reg_lambda': 1.0,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 0.8}

## Шаг 2. Подбираем параметры дерева
* **max_depth** - будем варьировать от 3 до 10 с шагом 2
* **min_child_weight** - от 1 до 6 с шагом 2

In [138]:
param_grid = {
    'max_depth': range(3, 10, 2),
    'min_child_weight': range(1, 6, 2)
}

clf = xgboost.XGBClassifier(**best_params_1) # в качестве отправной точки возьмём модель с наилучшими параметрами предыдущего шага

gs = GridSearchCV(clf, param_grid, scoring='accuracy', cv=cv, verbose=1)

gs.fit(X_train_i_s, y_train)
best_params_2 = gs.best_estimator_.get_params()
print('Best score (accuracy): ', gs.best_score_)
print('Best params: ')
best_params_2

Fitting 8 folds for each of 12 candidates, totalling 96 fits
Best score (accuracy):  0.83950617284
Best params: 


[Parallel(n_jobs=1)]: Done  96 out of  96 | elapsed:   14.6s finished


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.8,
 'gamma': 0.1,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 3,
 'missing': None,
 'n_estimators': 200,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0.0,
 'reg_lambda': 1.0,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 0.8}

## Шаг 3. Подбираем gamma (критерий создания поддерева)
* **gamma** - от 0 до 0.5 с шагом 0.1

In [139]:
param_grid = {
    'gamma': [0.1*i for i in range(6)]
}

clf = xgboost.XGBClassifier(**best_params_2)

gs = GridSearchCV(clf, param_grid, scoring='accuracy', cv=cv, verbose=0)

gs.fit(X_train_i_s, y_train)
best_params_3 = gs.best_estimator_.get_params()
print('Best score (accurac): ', gs.best_score_)
print('Best params: ')
best_params_3

Best score (accurac):  0.835016835017
Best params: 


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.8,
 'gamma': 0.30000000000000004,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 3,
 'missing': None,
 'n_estimators': 200,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0.0,
 'reg_lambda': 1.0,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 0.8}

## Шаг 4. Затюним subsample и colsample_bytree
* **subsample** - от 0.5 до 1.0 с шагом 0.1
* **colsample_bytree** - от 0.5 до 1.0 с шагом 0.1

In [140]:
param_grid = {
    'subsample': [0.5 + 0.1*i for i in range(6)],
    'colsample_bytree': [0.5 + 0.1*i for i in range(6)]
}

clf = xgboost.XGBClassifier(**best_params_3)

gs = GridSearchCV(clf, param_grid, scoring='accuracy', cv=cv, verbose=0)

gs.fit(X_train_i_s, y_train)
best_params_4 = gs.best_estimator_.get_params()
print('Best score (accuracy): ', gs.best_score_)
print('Best params: ')
best_params_4

Best score (accuracy):  0.837261503928
Best params: 


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.9,
 'gamma': 0.30000000000000004,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 3,
 'missing': None,
 'n_estimators': 200,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0.0,
 'reg_lambda': 1.0,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 1.0}

## Шаг 5. Регуляризация
* **reg_alpha** [1e-5, 1e-2, 0.1, 1, 100]
* **reg_lambda** [1e-5, 1e-2, 0.1, 1, 100]

In [141]:
param_grid = {
'reg_alpha': [1e-5, 1e-2, 0.1, 1, 100],
'reg_lambda': [1e-5, 1e-2, 0.1, 1, 100]
}

clf = xgboost.XGBClassifier(**best_params_4)

gs = GridSearchCV(clf, param_grid, scoring='accuracy', cv=cv, verbose=1)

gs.fit(X_train_i_s, y_train)
best_params_5 = gs.best_estimator_.get_params()
print('Best score (accuracy): ', gs.best_score_)
print('Best params: ')
best_params_5

Fitting 8 folds for each of 25 candidates, totalling 200 fits
Best score (accuracy):  0.838383838384
Best params: 


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:   16.0s finished


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.9,
 'gamma': 0.30000000000000004,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 3,
 'missing': None,
 'n_estimators': 200,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 1e-05,
 'reg_lambda': 0.1,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 1.0}

## Шаг 6. Learning rate
Чем меньше у нас **n_estimators** в ансамбле, тем быстрее нам нужно двигаться с каждым шагом (добавлением нового классификатора), т.е. делать больший **learning_rate**. Обычно **learning rate** варьируют так, чтобы произведение **n_estimators** x **learning_rate** оставалось инвариантным

In [142]:
clf = xgboost.XGBClassifier(**best_params_5)
clf.fit(X_train_i_s,y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.9,
       gamma=0.30000000000000004, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=3, missing=None, n_estimators=200,
       nthread=-1, objective='binary:logistic', reg_alpha=1e-05,
       reg_lambda=0.1, scale_pos_weight=0.6229508196721312, seed=0,
       silent=True, subsample=1.0)

In [143]:
clf.score (X_train_i_s, y_train)

0.88215488215488214

In [123]:
import numpy as np
best_params_6 = best_params_5.copy()
clf = xgboost.XGBClassifier(**best_params_6)
best_n_estimators = clf.get_params()['n_estimators'] # возьмём наилучшие значения n_estimators с предыдущего шага
best_learning_rate = best_params['learning_rate'] # аналогичная запись
invariant_composition = best_n_estimators * best_learning_rate
n_estimators_range = [10, 30, 100, 200, 400, 600, 800, 1000]

best_score = gs.best_score_ # возьмём наилучшее качество с предыдущего шага

for n_estimators in n_estimators_range:
    learning_rate = invariant_composition / n_estimators
    clf.set_params(n_estimators=n_estimators, learning_rate=learning_rate)
    #aucs = []
    accurs = []
    for train_idx, test_idx in cv.split(X_train_i_s):
        X_train_fold, X_test_fold = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_train_fold, y_test_fold = y_train.iloc[train_idx], y_train.iloc[test_idx]
        clf.fit(X_train_fold, y_train_fold)
        #preds = clf.predict_proba(X_test_fold)
        #auc = roc_auc_score(y_test_fold, preds[:, 1])
        #aucs.append(auc)
        accur = clf.score(X_test_fold, y_test_fold)
        accurs.append(accur)
    accur = np.mean(accurs)
    if accur > best_score:
        best_n_estimators = n_estimators
        best_learning_rate = learning_rate
        best_score = accur
        
best_params_6['n_estimators'] = best_n_estimators
best_params_6['learning_rate'] = best_learning_rate

print('Best score (accuracy): ', best_score)
print('Best params: ')
best_params_6

Best score (accuracy):  0.835016835017
Best params: 


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.7,
 'gamma': 0.0,
 'learning_rate': 0.025,
 'max_delta_step': 0,
 'max_depth': 7,
 'min_child_weight': 5,
 'missing': None,
 'n_estimators': 200,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 1e-05,
 'reg_lambda': 0.01,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 1.0}

In [124]:
clf = xgboost.XGBClassifier(**best_params_6)
clf.fit(X_train_i_s,y_train)
clf.score (X_train_i_s, y_train)

0.85970819304152635

# HyperOpt

In [160]:
hp.quniform('max_depth', 1, 10, 1)

In [ ]:
# в этой функции мы проверяем, как ведёт себя модель при заданных параметрах
def score(params):
    print("Training with params : ")
    print(params)
    params['max_depth'] = int(params['max_depth'])
    params['n_estimators'] = int(params['n_estimators'])
    clf = xgboost.XGBClassifier(**params)
    accurs = []
    # Для оценки качества используем KFold, который определили выше
    for train_idx, test_idx in cv.split(X_train_i_s):
        X_train_fold, X_test_fold = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_train_fold, y_test_fold = y_train.iloc[train_idx], y_train.iloc[test_idx]
        clf.fit(X_train_fold, y_train_fold)
        preds = clf.predict_proba(X_test_fold)
        #auc = roc_auc_score(y_test_fold, preds[:, 1])
        #aucs.append(auc)
        accur = clf.score(X_test_fold, y_test_fold)
        accurs.append(accur)
    accur = np.mean(accurs)
    result = {'loss': accur, 'status': STATUS_OK}
    #print(result)
    return result


# это наша главная функция, в которой мы задаём параметры
def optimize(trials):
    space = {
             'n_estimators' : hp.quniform('n_estimators', 100, 1000, 5), # (название параметра, от, до, шаг)
             'learning_rate' : hp.quniform('learning_rate', 0.025, 0.5, 0.025),
             'max_depth' : hp.quniform('max_depth', 1, 10, 1),
             'min_child_weight' : hp.quniform('min_child_weight', 1, 6, 1),
             'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
             'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
             'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05),
             'objective': 'reg:linear',
             'silent' : 1,
             'scale_pos_weight': hp.quniform('scale_pos_weight', 0.5, 10., 0.5),
             'reg_alpha': 0.0,
             'reg_lambda': 1.0
             }

    best = fmin( score, space, algo=tpe.suggest, trials=trials, max_evals=15000)
    print('Best: ')
    print(best)

#сюда будет записана
trials = Trials()

optimize(trials)

Training with params : 
{'colsample_bytree': 0.65, 'gamma': 0.65, 'learning_rate': 0.25, 'max_depth': 4.0, 'min_child_weight': 6.0, 'n_estimators': 925.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 1.5, 'silent': 1, 'subsample': 0.65}
Training with params : 
{'colsample_bytree': 0.8500000000000001, 'gamma': 0.8, 'learning_rate': 0.45, 'max_depth': 1.0, 'min_child_weight': 3.0, 'n_estimators': 820.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 7.5, 'silent': 1, 'subsample': 0.8500000000000001}
Training with params : 
{'colsample_bytree': 1.0, 'gamma': 0.8500000000000001, 'learning_rate': 0.5, 'max_depth': 4.0, 'min_child_weight': 5.0, 'n_estimators': 950.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 8.5, 'silent': 1, 'subsample': 0.9}
Training with params : 
{'colsample_bytree': 0.75, 'gamma': 0.7000000000000001, 'learning_rate': 0.45, 'max_depth': 4.0, 'min_child_we

Training with params : 
{'colsample_bytree': 1.0, 'gamma': 1.0, 'learning_rate': 0.275, 'max_depth': 1.0, 'min_child_weight': 1.0, 'n_estimators': 295.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 9.5, 'silent': 1, 'subsample': 0.75}
Training with params : 
{'colsample_bytree': 0.55, 'gamma': 0.9, 'learning_rate': 0.225, 'max_depth': 9.0, 'min_child_weight': 2.0, 'n_estimators': 100.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 9.0, 'silent': 1, 'subsample': 0.65}
Training with params : 
{'colsample_bytree': 0.9500000000000001, 'gamma': 0.9500000000000001, 'learning_rate': 0.30000000000000004, 'max_depth': 1.0, 'min_child_weight': 1.0, 'n_estimators': 440.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 5.5, 'silent': 1, 'subsample': 0.8500000000000001}
Training with params : 
{'colsample_bytree': 0.8, 'gamma': 1.0, 'learning_rate': 0.225, 'max_depth': 3.0, 'min_child

Training with params : 
{'colsample_bytree': 0.9500000000000001, 'gamma': 0.6000000000000001, 'learning_rate': 0.35000000000000003, 'max_depth': 1.0, 'min_child_weight': 5.0, 'n_estimators': 415.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 2.0, 'silent': 1, 'subsample': 0.7000000000000001}
Training with params : 
{'colsample_bytree': 0.9, 'gamma': 0.9500000000000001, 'learning_rate': 0.30000000000000004, 'max_depth': 4.0, 'min_child_weight': 6.0, 'n_estimators': 250.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 8.0, 'silent': 1, 'subsample': 0.8500000000000001}
Training with params : 
{'colsample_bytree': 0.8500000000000001, 'gamma': 0.5, 'learning_rate': 0.125, 'max_depth': 2.0, 'min_child_weight': 5.0, 'n_estimators': 625.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 9.5, 'silent': 1, 'subsample': 0.8}
Training with params : 
{'colsample_bytree': 0.75, 'gamma': 

Training with params : 
{'colsample_bytree': 0.75, 'gamma': 0.9500000000000001, 'learning_rate': 0.325, 'max_depth': 2.0, 'min_child_weight': 6.0, 'n_estimators': 105.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 6.5, 'silent': 1, 'subsample': 1.0}
Training with params : 
{'colsample_bytree': 0.55, 'gamma': 0.9500000000000001, 'learning_rate': 0.47500000000000003, 'max_depth': 2.0, 'min_child_weight': 6.0, 'n_estimators': 325.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 6.0, 'silent': 1, 'subsample': 1.0}
Training with params : 
{'colsample_bytree': 0.5, 'gamma': 0.8500000000000001, 'learning_rate': 0.375, 'max_depth': 1.0, 'min_child_weight': 6.0, 'n_estimators': 690.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 7.0, 'silent': 1, 'subsample': 0.9500000000000001}
Training with params : 
{'colsample_bytree': 0.75, 'gamma': 0.9, 'learning_rate': 0.30000000000000004,

Training with params : 
{'colsample_bytree': 0.65, 'gamma': 0.75, 'learning_rate': 0.45, 'max_depth': 7.0, 'min_child_weight': 5.0, 'n_estimators': 410.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 10.0, 'silent': 1, 'subsample': 0.9}
Training with params : 
{'colsample_bytree': 0.8500000000000001, 'gamma': 0.7000000000000001, 'learning_rate': 0.375, 'max_depth': 1.0, 'min_child_weight': 6.0, 'n_estimators': 195.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 6.0, 'silent': 1, 'subsample': 0.9500000000000001}
Training with params : 
{'colsample_bytree': 0.55, 'gamma': 1.0, 'learning_rate': 0.5, 'max_depth': 2.0, 'min_child_weight': 4.0, 'n_estimators': 430.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 8.0, 'silent': 1, 'subsample': 1.0}
Training with params : 
{'colsample_bytree': 0.5, 'gamma': 0.75, 'learning_rate': 0.07500000000000001, 'max_depth': 3.0, 'min_child_

...

Training with params : 
{'colsample_bytree': 0.5, 'gamma': 0.5, 'learning_rate': 0.025, 'max_depth': 1.0, 'min_child_weight': 1.0, 'n_estimators': 100.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 10.0, 'silent': 1, 'subsample': 0.8500000000000001}
Training with params : 
{'colsample_bytree': 0.5, 'gamma': 0.55, 'learning_rate': 0.025, 'max_depth': 1.0, 'min_child_weight': 4.0, 'n_estimators': 130.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 9.0, 'silent': 1, 'subsample': 0.9}
Training with params : 
{'colsample_bytree': 0.5, 'gamma': 0.55, 'learning_rate': 0.05, 'max_depth': 1.0, 'min_child_weight': 2.0, 'n_estimators': 100.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 9.5, 'silent': 1, 'subsample': 0.8500000000000001}
Training with params : 
{'colsample_bytree': 0.5, 'gamma': 0.5, 'learning_rate': 0.025, 'max_depth': 1.0, 'min_child_weight': 1.0, 'n_estimators':

Training with params : 
{'colsample_bytree': 0.5, 'gamma': 0.5, 'learning_rate': 0.025, 'max_depth': 1.0, 'min_child_weight': 1.0, 'n_estimators': 100.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 9.0, 'silent': 1, 'subsample': 0.9}
Training with params : 
{'colsample_bytree': 0.5, 'gamma': 0.55, 'learning_rate': 0.05, 'max_depth': 1.0, 'min_child_weight': 4.0, 'n_estimators': 115.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 9.5, 'silent': 1, 'subsample': 0.8500000000000001}
Training with params : 
{'colsample_bytree': 0.5, 'gamma': 0.55, 'learning_rate': 0.025, 'max_depth': 2.0, 'min_child_weight': 1.0, 'n_estimators': 130.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 9.5, 'silent': 1, 'subsample': 0.9}
Training with params : 
{'colsample_bytree': 0.55, 'gamma': 0.55, 'learning_rate': 0.025, 'max_depth': 1.0, 'min_child_weight': 4.0, 'n_estimators': 100.0, 'objec

Training with params : 
{'colsample_bytree': 0.55, 'gamma': 0.55, 'learning_rate': 0.05, 'max_depth': 1.0, 'min_child_weight': 4.0, 'n_estimators': 100.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 9.5, 'silent': 1, 'subsample': 0.8500000000000001}
Training with params : 
{'colsample_bytree': 0.5, 'gamma': 0.5, 'learning_rate': 0.025, 'max_depth': 1.0, 'min_child_weight': 2.0, 'n_estimators': 115.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 9.5, 'silent': 1, 'subsample': 0.8500000000000001}
Training with params : 
{'colsample_bytree': 0.55, 'gamma': 0.55, 'learning_rate': 0.05, 'max_depth': 1.0, 'min_child_weight': 1.0, 'n_estimators': 145.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 9.5, 'silent': 1, 'subsample': 0.8500000000000001}
Training with params : 
{'colsample_bytree': 0.5, 'gamma': 0.55, 'learning_rate': 0.025, 'max_depth': 1.0, 'min_child_weight': 4.0,

Training with params : 
{'colsample_bytree': 0.5, 'gamma': 0.5, 'learning_rate': 0.025, 'max_depth': 1.0, 'min_child_weight': 4.0, 'n_estimators': 120.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 10.0, 'silent': 1, 'subsample': 0.8500000000000001}
Training with params : 
{'colsample_bytree': 0.55, 'gamma': 0.55, 'learning_rate': 0.05, 'max_depth': 1.0, 'min_child_weight': 4.0, 'n_estimators': 100.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 9.5, 'silent': 1, 'subsample': 0.9}
Training with params : 
{'colsample_bytree': 0.55, 'gamma': 0.55, 'learning_rate': 0.025, 'max_depth': 1.0, 'min_child_weight': 1.0, 'n_estimators': 130.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 10.0, 'silent': 1, 'subsample': 0.8500000000000001}
Training with params : 
{'colsample_bytree': 0.5, 'gamma': 0.55, 'learning_rate': 0.05, 'max_depth': 1.0, 'min_child_weight': 4.0, 'n_estimator

Training with params : 
{'colsample_bytree': 0.5, 'gamma': 0.55, 'learning_rate': 0.05, 'max_depth': 1.0, 'min_child_weight': 1.0, 'n_estimators': 145.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 9.5, 'silent': 1, 'subsample': 0.9}
Training with params : 
{'colsample_bytree': 0.55, 'gamma': 0.55, 'learning_rate': 0.025, 'max_depth': 1.0, 'min_child_weight': 4.0, 'n_estimators': 100.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 10.0, 'silent': 1, 'subsample': 0.8500000000000001}
Training with params : 
{'colsample_bytree': 0.5, 'gamma': 0.5, 'learning_rate': 0.025, 'max_depth': 1.0, 'min_child_weight': 1.0, 'n_estimators': 120.0, 'objective': 'reg:linear', 'reg_alpha': 0.0, 'reg_lambda': 1.0, 'scale_pos_weight': 9.5, 'silent': 1, 'subsample': 0.9}
Training with params : 
{'colsample_bytree': 0.55, 'gamma': 0.55, 'learning_rate': 0.05, 'max_depth': 1.0, 'min_child_weight': 4.0, 'n_estimators': 100.0, 'obje

In [ ]:
best_params_7 = trials.best_trial['misc']['vals']
best_params_7

In [194]:
best_params_7 = trials.best_trial['misc']['vals']
#best_params_7['colsample_bytree'] = int(best_params_7['colsample_bytree'][0])
#best_params_7['gamma'] = int(best_params_7['gamma'][0])
#best_params_7['learning_rate'] = int(best_params_7['learning_rate'][0])
#best_params_7['max_depth'] = int(best_params_7['max_depth'][0])
#best_params_7['min_child_weight'] = int(best_params_7['min_child_weight'][0])
#best_params_7['n_estimators'] = int(best_params_7['n_estimators'][0])
#best_params_7['scale_pos_weight'] = int(best_params_7['scale_pos_weight'][0])
#best_params_7['subsample'] = int(best_params_7['subsample'][0])
best_params_7

{'colsample_bytree': 0,
 'gamma': 0,
 'learning_rate': 0,
 'max_depth': 1,
 'min_child_weight': 4,
 'n_estimators': 610,
 'scale_pos_weight': 10,
 'subsample': 1}

In [195]:
clf = xgboost.XGBClassifier(**best_params_7)
clf.fit(X_train_i_s,y_train)
clf.score (X_train_i_s, y_train)

XGBoostError: b'[04:28:11] src/tree/updater_colmaker.cc:161: Check failed: (n) > (0) colsample_bytree=0 is too small that no feature can be included'

Создаем сабмит

In [157]:
clf = xgboost.XGBClassifier(**best_params_6)
clf.fit(X_train_i_s,y_train)
y_test = clf.predict(X_test_i_s)
submit = pd.DataFrame(test.PassengerId)
submit['Survived'] = y_test
submit.tail()
submit.to_csv('submit.csv', index = False)